In [1]:
import numpy as np
import random
import plotly.graph_objects as go

In [4]:
def yinshe(dic, x):
    while x in dic.keys():
        x = dic[x]
    return x

def jiaocha(Lis1, Lis2):
    lis1 = Lis1.copy()
    lis2 = Lis2.copy()
    n = len(lis1)
    cross_points_1 = np.random.randint(low=0, high=n-1)
    cross_points_2 = np.random.randint(low=cross_points_1+1, high=n)
    
    yinshe2 = dict(zip(lis1[cross_points_1: cross_points_2],lis2[cross_points_1: cross_points_2]))
    yinshe1 = dict(zip(lis2[cross_points_1: cross_points_2],lis1[cross_points_1: cross_points_2]))

    lis1[cross_points_1: cross_points_2], lis2[cross_points_1: cross_points_2] = lis2[cross_points_1: cross_points_2], lis1[cross_points_1: cross_points_2]
    
    lis1[:cross_points_1] = list(map(lambda x: yinshe(yinshe1, x), lis1[:cross_points_1]))
    lis1[cross_points_2:] = list(map(lambda x: yinshe(yinshe1, x), lis1[cross_points_2:]))
    lis2[:cross_points_1] = list(map(lambda x: yinshe(yinshe2, x), lis2[:cross_points_1]))
    lis2[cross_points_2:] = list(map(lambda x: yinshe(yinshe2, x), lis2[cross_points_2:]))
    return lis1, lis2

def mutation(pop, MUTATION_RATE=0.003):
    if np.random.rand() < MUTATION_RATE:
        n = len(pop)
        cross_points_1 = np.random.randint(low=0, high=n-1)
        cross_points_2 = np.random.randint(low=cross_points_1+1, high=n)
        pop[cross_points_1], pop[cross_points_2] = pop[cross_points_2], pop[cross_points_1]
    return pop

def initpop(N, pop):
    def lis_shuffle(lis):
        lisc = lis.copy()
        random.shuffle(lisc)
        return lisc
    pops = [pop]*N
    pops = list(map(lis_shuffle, pops))
    return pops

def dis(start, end):
    # 两点之间距离
    return np.sqrt((end[0]-start[0])**2+(end[1]-start[1])**2)

def fuc(x):
    dis_sum = dis((0,0),local[x[0]])
    for i in range(1,len(x)):
        dis_sum += dis(local[x[i-1]],local[x[i]])
    dis_sum += dis(local[x[-1]],(0,0))
    return dis_sum

def get_fitness(pops):
    return list(map(fuc,pops))

def select(pops):
    k = round(np.sqrt(len(pops)+0.25)+0.5)
    fitness = get_fitness(pops)
    father_pops = []
    for i in range(k):
        min_index = np.array(fitness).argmin()
        father_pops.append(pops.pop(min_index))
        fitness.pop(min_index)
    return father_pops

def evolution(pops):
    father_pops = select(pops)
    k = len(father_pops)
    new_pops = []
    for i in range(k-1):
        for j in range(i+1,k):
            son1, son2 = jiaocha(father_pops[i], father_pops[j])
            son1 = mutation(son1, MUTATION_RATE=0.3)
            son2 = mutation(son2, MUTATION_RATE=0.3)
            new_pops.append(son1)
            new_pops.append(son2)
    return new_pops

def out_xy(pop, local):
    x, y = [0], [0]
    for i in pop:
        x.append(local[i][0])
        y.append(local[i][1])
    x.append(0)
    y.append(0)
    return x, y

In [20]:
frames = []
N = 100

local = [(-5, -2), (-3, 4), (4, 5), (-4, -3), (-1, 1), (0, -1), (1, 3), (2, -4), (-2, 0), (3, 2)]

pops = initpop(N, list(range(10)))
for _ in range(50):
    pops = evolution(pops)
    x, y = out_xy(pops[np.array(get_fitness(pops)).argmin()], local)
    frames.append(go.Frame(data=[go.Scatter(x=x, y=y, mode="lines+markers", marker=dict(size=10))], layout=go.Layout(title_text=f"当前最短路径长度为：{round(min(get_fitness(pops)), 2)}")))
frames[-1] = go.Frame(data=[go.Scatter(x=x, y=y, mode="lines+markers", marker=dict(size=10))], layout=go.Layout(title_text=f"遗传结束，最短路径长度为：{round(min(get_fitness(pops)), 2)}"))

fig = go.Figure(
    data=[go.Scatter(x=x, y=y, mode="markers", marker=dict(size=10)), go.Scatter(x=[0], y=[0], mode="markers", marker=dict(color="red", size=15), name="起点")],
    layout=go.Layout(
        xaxis=dict(range=[-6, 6], autorange=False),
        yaxis=dict(range=[-6, 6], autorange=False),
        title="点击演示按钮",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="演示",
                          method="animate",
                          args=[None])])]
    ),
    frames=frames
)

fig.update_layout(
    width=800,
    height=800,
)


fig.show()